In [3]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [4]:
class SyntheticTimeSeries(Dataset):
    """
    Genera datos sintéticos donde cada muestra es una serie de longitud seq_len
    y cada timestep tiene 80 características numéricas.
    """
    def __init__(self, n_samples=2000, seq_len=50, input_dim=80, num_classes=2):
        super().__init__()
        self.seq_len = seq_len
        self.input_dim = input_dim
        self.num_classes = num_classes
        
        # Generamos X aleatorios: (n_samples, seq_len, input_dim)
        self.X = torch.randn(n_samples, seq_len, input_dim)
        
        # Generamos etiquetas Y: (n_samples,)
        self.Y = torch.randint(0, num_classes, (n_samples,))
    
    def __len__(self):
        return self.X.size(0)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [8]:
# Parámetros del dataset sintético
n_samples  = 2000
seq_len    = 50
input_dim  = 80   
num_classes= 2
    
dataset = SyntheticTimeSeries(
    n_samples=n_samples,
    seq_len=seq_len,
    input_dim=input_dim,
    num_classes=num_classes
)

In [9]:
# Obtenemos la primera instancia (x=features, y=label)
first_x, first_y = dataset[0]

# Imprimimos en consola la forma y los datos
print("Shape de la primera instancia:", first_x.shape)
print("Datos de la primera instancia (X):\n", first_x)
print("Etiqueta (Y):", first_y)

Shape de la primera instancia: torch.Size([50, 80])
Datos de la primera instancia (X):
 tensor([[-1.6547,  1.0761,  0.1210,  ...,  0.6420,  0.2255,  0.7914],
        [-1.7789,  0.0633, -1.0834,  ...,  0.9964,  1.3432,  1.1256],
        [ 0.3281, -0.2124,  1.7209,  ..., -0.8452, -0.1610,  0.4863],
        ...,
        [-0.3591, -1.7466, -0.9314,  ..., -1.0725, -0.3175, -0.1341],
        [-0.1022, -0.7715, -0.4931,  ...,  1.2214, -0.0157, -0.0594],
        [ 0.2476, -0.6136, -0.0281,  ...,  0.9709, -0.6975, -0.3212]])
Etiqueta (Y): tensor(1)
